In [1]:
import torch
import os
import argparse
import pandas as pd

/home/ubuntu/anaconda3/envs/jeonghyeon/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
from module.model import model as tagger
from module.model import device
from module.model import tokenizer
from module.model import id2label

Some weights of the model checkpoint at monologg/koelectra-base-v3-discriminator were not used when initializing ElectraModel: ['discriminator_predictions.dense.bias', 'discriminator_predictions.dense_prediction.bias', 'discriminator_predictions.dense_prediction.weight', 'discriminator_predictions.dense.weight']
- This IS expected if you are initializing ElectraModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing ElectraModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [3]:
# parser = argparse.ArgumentParser()
# parser.add_argument("-q", "--query", dest="keyword", action="store")
# args = parser.parse_args()

## Loading

In [131]:
file_list = os.listdir('./inference_data')
file_path = []
for file_name in file_list:
    tmp_path = os.path.join('./inference_data', file_name)
    file_path.append(tmp_path)
file_path.sort()

In [132]:
load_df = lambda x: pd.read_csv(x)
dfs = [load_df(x) for x in file_path]
data = dfs[0]
data.dropna(inplace=True)
# data[pd.isna(data).any(axis=1)]
# data.isna().nunique()
print(f'length of {file_path[0]}: {len(data)}')

length of ./inference_data/event_port.csv: 171


In [133]:
data.isnull().any()

Event    False
dtype: bool

## Cleansing

In [134]:
# def preprocess(df):
#     df['Event'] = df['Event'].str.replace("．", ".", regex=False)
#     df['Event'] = df['Event'].astype(str)
#     # df['Event'] = df['Event'].str.replace(r'[^ㄱ-ㅣ가-힣0-9a-zA-Z.]+', " ", regex=True)
#     return df

In [135]:
# data = preprocess(data)

In [136]:
keyword = '화물'

data['Indexes'] = data.Event.str.find(keyword)
infer_data = data[data.Indexes > -1]
# infer_data

In [137]:
# for event in infer_data.Event:
#     print(event)
#     print()

In [138]:
infer_data_lst = infer_data.Event.apply(lambda x: tokenizer(x, max_length=256, padding='max_length', truncation=True))
input_ids_lst = []
input_masks_lst = []
for e in infer_data_lst:
    input_ids_lst.append(e['input_ids'])
    input_masks_lst.append(e['attention_mask'])
len(input_ids_lst), len(input_masks_lst)

(36, 36)

In [205]:
### for custom test

test = ['2014년 12월 27일(토) 20시50분경 전남 광양시 소재 OO부두에서 파이프 선적작업 중 OO(주)소속 피재자가 선박 상부 해치커버 위에서 도구박스를 선창 내부로 내리던 중 몸의 중심을 잃고 선창 내부로 떨어져 사망', '2014년 11월 4일(화) 20시30분경 인천광역시 중구 소재 OO부두에서 컨테이너 선적작업을 하던 피재자가 해치코밍 상단에서 무전을 하며 작업지휘를 하다가 중심을 잃고 선창 구조물 바닥으로 떨어져 사망', '"한양에이스호는 2015년 1월 11일 7시 40분경 울산항 제4부두 1번선에 접안하였고 육상의 검사업체가 월워시테스트(Wall Wash Test)를 하였음. 월워시테스트 통과 후 09시 45분경부터 혼산(Mixed Acid) 선적작업을 시작함. 같은 날 14시 39분경 갑자기 2번 우현 탱크 쪽이 들썩이며 순간적인 폭발 일어남. 약 2초 후에는 좌현 쪽 갑판도 들썩이며 폭발이 발생. 폭발 후, 출동한 소방차와 구급차의 도움으로 선원들은 모두 육상으로 대피하였으나, 선원 4명이 증기에 노출되어 중상 1명, 경상 3명이 발생. 혼산의 일부는 바다로 흘러 들어감."']
test = pd.Series(test)
test

test_lst = test.apply(lambda x: tokenizer(x, max_length=256, padding='max_length', truncation=True))
input_ids_lst = []
input_masks_lst = []
for e in test_lst:
    input_ids_lst.append(e['input_ids'])
    input_masks_lst.append(e['attention_mask'])
len(input_ids_lst), len(input_masks_lst)

(3, 3)

In [206]:
def infer(input_ids_lst, input_masks_lst):

    tagger.eval()
    with torch.no_grad():
        infer_results = []
        for input_ids, input_masks in zip(input_ids_lst, input_masks_lst): 
            input_tensor = torch.LongTensor(input_ids).unsqueeze(0).to(device)
            mask_tensor = torch.LongTensor(input_masks).unsqueeze(0).to(device)
            output = tagger(input_tensor, mask_tensor)
            pred = torch.argmax(output, dim=-1).squeeze().detach().cpu().tolist()
            infer_results.append(pred)

        return infer_results
    
infer_results = infer(input_ids_lst, input_masks_lst)

In [207]:
subwordsList = []
tagsList = []
for input_ids, result in zip(input_ids_lst, infer_results):
    # print(input_ids[1:])
    subwords = [x for x in input_ids if x != 0][1:-1]
    # print(subwords)
    subwords = list(map(lambda x: tokenizer.convert_ids_to_tokens(x), subwords))
    subwordsList.append(subwords)
    lenSubwords = len(subwords)    

    result = result[1:lenSubwords+1]
    tags = []
    for id in result:
        tags.append(id2label[id])
    tagsList.append(tags)

In [208]:
for subwords, tags in zip(subwordsList, tagsList):
    for subword, tag in zip(subwords, tags):
        print(subword, tag)
    print()
    

201 DAT_B
4 DAT_I
년 DAT_I
12 DAT_I
##월 DAT_I
27 DAT_I
##일 DAT_I
( DAT_I
토 DAT_I
) TIM_B
20 TIM_B
##시 TIM_I
##50 TIM_I
##분 TIM_I
##경 TIM_I
전남 LOC_B
광양시 LOC_I
소재 LOC_I
OO LOC_I
부두 LOC_I
에서 LOC_I
파이프 WRK_B
선적작업 WRK_I
중 WRK_I
OO LOC_B
( LOC_I
주 LOC_I
) LOC_I
소속 -
피재자가 LOC_I
선박 LOC_I
상부 LOC_I
해치 LOC_I
커버 LOC_I
위 LOC_I
##에 LOC_I
##서 LOC_I
도구 AFW_B
박스 AFW_I
를 AFW_I
선창 -
내부 -
##로 -
내리 -
##던 -
중 -
몸 ANM_B
##의 ANM_I
중 -
심 -
##을 -
잃 -
##고 -
선창 -
내부 -
##로 -
떨어져 -
사망 -

201 DAT_B
4 DAT_I
년 DAT_I
11 DAT_I
##월 DAT_I
4 DAT_I
일 DAT_I
( DAT_I
화 DAT_I
) TIM_B
20 TIM_I
##시 TIM_I
##30 TIM_I
##분 TIM_I
##경 TIM_I
인천광역 LOC_B
시 LOC_I
중 LOC_I
구 LOC_I
소재 LOC_I
OO LOC_I
부두 LOC_I
에서 LOC_I
컨테이너 WRK_B
선적작업 WRK_I
을 WRK_I
하 -
##던 -
피재자가 AFW_B
해치 AFW_I
##코 AFW_I
##밍 AFW_I
상단 -
##에 -
##서 -
무전 -
##을 -
하 -
##며 -
작업 -
지휘 -
##를 -
하 -
##다가 -
중 -
심 -
##을 -
잃 -
##고 -
선창 AFW_B
구조물 -
바닥 -
##으로 -
떨어져 -
사망 -

" -
한양에이스호 -
는 -
2015 DAT_B
##년 DAT_I
1 DAT_I
##월 DAT_I
11 DAT_I
##일 DAT_I
7 TIM_B
##시 TIM_I
4 TIM_I
0 TIM_I
##분 TIM_I
##경 T

In [209]:
### v2

# 날짜(DAT), 시간(TIME), 장소(LOC), 작업 정보(WRK)를 추출해서 txt 파일에 저장

def extract(tagtype, tags, subwords):
    tag_start = f'{tagtype}_B'
    tags_idx_dic = {i:x for i,x in zip(range(len(tags)), tags) if x == tag_start}
    tag_start_ids = list(tags_idx_dic.keys())
    if tag_start_ids:
        taggedList = []
        for b_idx in tag_start_ids:
            for i, tag in enumerate(tags[b_idx:]):
                if i > 0 and tag != f'{tagtype}_I':
                    tagged = subwords[b_idx:b_idx+i]
                    tagged = tokenizer.convert_tokens_to_string(tagged)
                    taggedList.append(tagged)
                    break
        if tagtype == 'DAT' or tagtype == 'TIM':
            taggedList = [x for x in taggedList if not len(x) < 2]
            taggedList = [taggedList[0]]
        return ' [SEP] '.join(taggedList)
    else:
        msg = f'{tagtype} not found.'
        return msg        

tagtypes = dict(
    dat='DAT',
    tim='TIM',
    loc='LOC',
    wrk='WRK',    
)

# subwordsList, tagsList
nerResults = []
count = 0
for subwords, tags in zip(subwordsList, tagsList):
    extracted = []
    for tagtype in tagtypes.values():
        tagged = extract(tagtype, tags, subwords)
        extracted.append(tagged)
    nerResults.append(extracted)
    count += 1

In [210]:
for i, e in enumerate(nerResults):
    print(e)
    if i == 300:
        break

['201 4 년 12월 27일 ( 토', '20시50분경', '전남 광양시 소재 OO 부두 에서 [SEP] OO ( 주 )', '파이프 선적작업 중']
['201 4 년 11월 4 일 ( 화', ') 20시30분경', '인천광역 시 중 구 소재 OO 부두 에서', '컨테이너 선적작업 을']
['2015년 1월 11일', '7시 4 0분경', '울산항 제 4 부두 1번선에', '혼산 ( [SEP] M ixed Acid ) 선적작업 을']


In [211]:
df = pd.DataFrame(nerResults, columns=['Date', 'Time', 'Location', 'Work'])

In [212]:
df[['Date', 'Time']]

,Date,Time
0,201 4 년 12월 27일 ( 토,20시50분경
1,201 4 년 11월 4 일 ( 화,) 20시30분경
2,2015년 1월 11일,7시 4 0분경


In [38]:
df

,Date,Time,Location,Work
0,2013년 2월 일 11,: 30경 부,○○,수행하던 위에서 를 하
1,2012년 11월 28일,TIM not found.,항 1 에서 화물 상,· 하차 에 연결된 를
2,DAT not found.,TIM not found.,LOC not found.,산화물 ( 광물 ) 을
3,2012 . 6 . 7 .,01 : 10경,일본 입항,WRK not found.
4,DAT not found.,TIM not found.,차 정박,WRK not found.
5,##일 15 : 30 을 출항,( 승무원 16,→,"대 , 화물 1 톤 ) 대 고장"
6,201 년 12월 2 일 ( 수 ) 10,TIM not found.,OO 근처에 한 에서 OO 소속,None
7,2012년 12월 10일,TIM not found.,항 60 에서 이 조작,으로 화물이 한쪽으로
8,2015년 12월 1일,22시 30분경,○○ 에 하여 (,에서
9,2015년 12월 10일,TIM not found.,"항 ○○ 원목 에서 원목 , 가",화물차를 오라고


In [39]:
df['Date'], df['Time'], df['Location'], df['Work'] = df['Date']+'\t', '\t'+df['Time']+'\t', '\t'+df['Location']+'\t', '\t'+df['Work']
df.to_csv('./inference_result.txt', index=False, sep='|')